In [1]:
import ee
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import geemap

In [2]:
# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-vertifysaptarshi')

In [3]:
# Load the feature collections
# table = ee.FeatureCollection("projects/ee-vertifysaptarshi/assets/TelanganaSoilTesting_V2")
table = ee.FeatureCollection("projects/ee-vertifysaptarshi/assets/icraf_ph")
telangana = ee.FeatureCollection("projects/ee-vertifysaptarshi/assets/telangana_district")

In [4]:
# Define the image collection and filter by date
dataset = ee.ImageCollection('COPERNICUS/S2') \
          .filterDate('2023-05-01', '2023-06-01')

In [5]:
gdf = geemap.ee_to_gdf(table)

In [6]:
# Define function to categorize pH values into custom classes
def categorize_ph(ph):
    if ph < 4.5:
        return 1
    elif 4.5 <= ph <= 5.9:
        return 2
    elif 6.0 <= ph <= 6.9:
        return 3
    elif 7.0 <= ph <= 7.8:
        return 4
    else:
        return 5

# Apply the function to create the 'PH Category' column
gdf['PH Category'] = gdf['PH'].apply(categorize_ph)

# Display the GeoDataFrame
# print(gdf)

# Create a dictionary for integer to string mapping
category_mapping = {
    1: 'Very Acidic',
    2: 'Moderately Acidic',
    3: 'Slightly Acidic',
    4: 'Neutral',
    5: 'Alkaline'
}

print("\nDictionary for integer to string mapping:")
print(category_mapping)



Dictionary for integer to string mapping:
{1: 'Very Acidic', 2: 'Moderately Acidic', 3: 'Slightly Acidic', 4: 'Neutral', 5: 'Alkaline'}


In [7]:
gdf['PH Category'].unique()

array([4, 5, 3, 2, 1])

In [8]:
gdf['PH Category'].value_counts()

2    62
3    44
4    25
5    13
1     1
Name: PH Category, dtype: int64

In [9]:
table = geemap.gdf_to_ee(gdf)

In [10]:
# Function to apply scaling factors
def apply_scale_factors(image):
    optical_bands = image.select(['B4', 'B3', 'B2']).multiply(0.0001)
    return image.addBands(optical_bands, None, True)

# Map the scaling function to the image collection
dataset = dataset.map(apply_scale_factors)

# Visualization parameters
visualization = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0.0,
    'max': 0.3,
}

# Create a mosaic of the image collection
l8_img = dataset.mosaic()

In [11]:
# Extract bands
green = l8_img.select('B3').toFloat()
red = l8_img.select('B4').toFloat()
nir = l8_img.select('B8').toFloat()  # NIR band in Sentinel-2

# Prepare the indices
SI1 = green.multiply(red).sqrt().rename('SI1')
SI2 = green.pow(2).add(red.pow(2)).add(nir.pow(2)).sqrt().rename('SI2')
NDSI = nir.subtract(red).divide(nir.add(red)).rename('NDSI')
BI = red.pow(2).add(nir.pow(2)).sqrt().rename('BI')
NDVI = l8_img.normalizedDifference(['B8', 'B4']).rename('NDVI')  # NIR and Red bands in Sentinel-2
SRSI = NDVI.subtract(1).pow(2).add(SI1.pow(2)).sqrt().rename('SRSI')

l8_img = l8_img.addBands([SI1, SI2, NDSI, BI, SRSI])
l8_img = l8_img.select('SI1', 'SI2', 'NDSI', 'BI', 'SRSI')

In [12]:
# Sample regions
training_data = l8_img.sampleRegions(
    collection=table,
    # properties=['Sample No', 'Soil pH', 'Soil pH Result'], #For WRMS
    properties=['Sample No', 'PH Category'], #For ICRAF
    scale=10
)

# Define the bands for regression
bands = [
    'SI1',
    'SI2',
    'NDSI',
    'BI',
    'SRSI'
    ]

# Define the target variable
target = 'PH Category'

In [13]:
# Split data into training and validation sets
split = 0.75
training_partition = training_data.randomColumn('random').filter(ee.Filter.lt('random', split))
validation_partition = training_data.randomColumn('random').filter(ee.Filter.gte('random', split))

In [14]:
# Convert training data to a pandas DataFrame
training_df = geemap.ee_to_df(training_data)

# Display the DataFrame
training_df.head(10)

,BI,NDSI,PH Category,SI1,SI2,SRSI
0,3193.000011,0.999835,4,0.257059,3193.000021,0.257059
1,3336.000010,0.999848,4,0.250123,3336.000019,0.250123
2,3188.000011,0.999832,4,0.259470,3188.000021,0.259471
3,3021.000011,0.999832,4,0.249658,3021.000021,0.249658
4,3105.000010,0.999838,4,0.248083,3105.000020,0.248083
5,3061.000011,0.999831,4,0.252996,3061.000021,0.252996
6,3193.000011,0.999834,5,0.258179,3193.000021,0.258179
7,3322.000009,0.999857,3,0.234274,3322.000017,0.234274
8,3379.000010,0.999850,2,0.247679,3379.000018,0.247679
9,3171.000010,0.999838,2,0.248226,3171.000019,0.248226


In [15]:
# Train the regressor
regressor = ee.Classifier.smileGradientTreeBoost(
    loss='LeastSquares',
    numberOfTrees=50,
    seed=20
)
regressor = ee.Classifier.smileRandomForest(numberOfTrees=80,
                                             seed=20)
# Train the model
trained_regressor = regressor.train(
    features=training_partition,
    classProperty=target,
    inputProperties=bands
)

In [16]:
# Uncomment to view the importances
# trained_regressor.explain()

In [17]:
# Classify the training and validation partitions
train = training_partition.classify(trained_regressor)
val = validation_partition.classify(trained_regressor)

In [18]:
# Convert training data to a pandas DataFrame
training_df = geemap.ee_to_df(train)

# Display the Unique targets
training_df['PH Category'].value_counts()

2    42
3    30
4    18
5    11
Name: PH Category, dtype: int64

In [19]:
# Convert training data to a pandas DataFrame
validation_df = geemap.ee_to_df(val)

# Display the DataFrame
validation_df['PH Category'].value_counts()

2    20
3    14
4     7
5     2
1     1
Name: PH Category, dtype: int64

In [20]:
from sklearn.metrics import classification_report, confusion_matrix

# Extract actual and predicted values for training and validation data
train_actual = training_df[target]
train_predicted = training_df['classification']
val_actual = validation_df[target]
val_predicted = validation_df['classification']

# Print classification report and confusion matrix for training data
print("Training Data Metrics:")
print(classification_report(train_actual, train_predicted))
print("Confusion Matrix:")
print(confusion_matrix(train_actual, train_predicted))

# Print classification report and confusion matrix for validation data
print("\nValidation Data Metrics:")
print(classification_report(val_actual, val_predicted))
print("Confusion Matrix:")
print(confusion_matrix(val_actual, val_predicted))


Training Data Metrics:
              precision    recall  f1-score   support

           2       1.00      0.98      0.99        42
           3       0.97      1.00      0.98        30
           4       1.00      1.00      1.00        18
           5       1.00      1.00      1.00        11

    accuracy                           0.99       101
   macro avg       0.99      0.99      0.99       101
weighted avg       0.99      0.99      0.99       101

Confusion Matrix:
[[41  1  0  0]
 [ 0 30  0  0]
 [ 0  0 18  0]
 [ 0  0  0 11]]

Validation Data Metrics:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
           2       0.53      0.80      0.64        20
           3       1.00      0.29      0.44        14
           4       0.43      0.43      0.43         7
           5       0.00      0.00      0.00         2

    accuracy                           0.52        44
   macro avg       0.39      0.30      0.30        44
wei

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
import time

# Sample classifier (replace this with your actual classifier)
classifier = trained_regressor

# Define the export parameters
description = 'RF_Model_ph'
assetId = 'projects/ee-vertifysaptarshi/assets/RF_Model_ph'
priority = 10

# Create an export classifier task to run.
# asset_id = 'projects/ee-vertifysaptarshi/assets/RF_Model_ph' # <> modify these
export_task = ee.batch.Export.classifier.toAsset(
    classifier=trained_regressor,
    description=description,
    assetId=assetId
    )

# Wait for the export task to complete
print('Exporting classifier...')
print(export_task.status())
export_task.start()
# Check the status of the export task
while export_task.active():
    print('Export task is still active...')
    time.sleep(10)
    print(export_task.status())

# Once the task is completed, print the final status
print('Classifier export completed:', export_task.status())

Exporting classifier...
{'state': <State.UNSUBMITTED: 'UNSUBMITTED'>}
Export task is still active...
{'state': 'FAILED', 'description': 'RF_Model_ph', 'priority': 100, 'creation_timestamp_ms': 1710837201677, 'update_timestamp_ms': 1710837211938, 'start_timestamp_ms': 1710837211577, 'task_type': 'EXPORT_CLASSIFIER', 'attempt': 1, 'error_message': "Cannot overwrite asset 'projects/ee-vertifysaptarshi/assets/RF_Model_ph'.", 'id': 'LX36YB5G3EV2MIW2TM2CDBLB', 'name': 'projects/ee-vertifysaptarshi/operations/LX36YB5G3EV2MIW2TM2CDBLB'}
Classifier export completed: {'state': 'FAILED', 'description': 'RF_Model_ph', 'priority': 100, 'creation_timestamp_ms': 1710837201677, 'update_timestamp_ms': 1710837211938, 'start_timestamp_ms': 1710837211577, 'task_type': 'EXPORT_CLASSIFIER', 'attempt': 1, 'error_message': "Cannot overwrite asset 'projects/ee-vertifysaptarshi/assets/RF_Model_ph'.", 'id': 'LX36YB5G3EV2MIW2TM2CDBLB', 'name': 'projects/ee-vertifysaptarshi/operations/LX36YB5G3EV2MIW2TM2CDBLB'}
